<a href="https://colab.research.google.com/github/kashyap333/Playground_Series_Season_3-_Episode_3/blob/main/Playground_Series_Season_3%2C_Episode_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c playground-series-s3e3


In [ ]:
!unzip playground-series-s3e3

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.gaussian_process import GaussianProcessClassifier
from pandas import get_dummies

In [75]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train.select_dtypes(include='object').columns

In [ ]:
train.select_dtypes(include='int').columns

In [66]:
y = train['Attrition']
X = train.drop('Attrition', axis=1)

In [67]:
def preprocessing(data):
  'Pass dataframe with no Attrition column'

  data.drop('id', axis=1, inplace = True)
  for col in data.columns:
    if len(data[col].unique()) < 2:
      data.drop(col, axis=1, inplace = True)

  data = pd.get_dummies(data)
  X = pd.DataFrame(StandardScaler().fit_transform(data), index=data.index, columns=data.columns)
      
  return X
  

In [68]:
X_train = preprocessing(X)

In [69]:
y_train = preprocessing(test)

###Encoding

In [28]:
X_train, X_test, y_train, y_test = train_test_split(train_data, train_target, test_size = 0.1, random_state=42)

In [70]:
Gaussian = GaussianProcessClassifier(random_state=0).fit(X_train, y)

In [71]:
Gaussian.score(X_train, y)

1.0

In [72]:
y_pred = Gaussian.predict_proba(y_train)

In [73]:
y_pred[:, 0]

array([0.50000044, 0.50014063, 0.50000023, ..., 0.50000008, 0.50000001,
       0.50000001])

In [76]:
result = pd.concat([pd.DataFrame(test['id']), pd.DataFrame(y_pred[:, 0])], axis=1)
result = result.rename(columns = {0: 'Attrition'})

In [221]:
result

,id,Attrition
0,1677,0.500015
1,1678,0.500327
2,1679,0.500026
3,1680,0.500129
4,1681,0.500285
...,...,...
1114,2791,0.500002
1115,2792,0.500015
1116,2793,0.500060
1117,2794,0.500043


In [77]:
result.to_csv('result.csv', index=False)

In [78]:
!kaggle competitions submit playground-series-s3e3 -f result.csv -m "My submission_1"

100% 26.1k/26.1k [00:00<00:00, 136kB/s]
Successfully submitted to Playground Series Season 3, Episode 3